## 필요한 라이브러리 불러오기 및 상수 선언

In [1]:
import pandas as pd
import glob
import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
# 앞뒤로 각각 몇 개의 데이터와 비교할지
NUM_NEIGHBORS = 5

## 데이터셋 가져오기

In [3]:
# PATH 설정
path = r'C:\Users\sungh\.vscode\repository\thumbnails\Thumbnails-Project\datasets\thumbnail_csv'

In [8]:
# dataset 하나로 합치기
all_files = glob.glob(os.path.join(path, "*.csv"))

In [9]:
df_all = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

In [10]:
# index 초기화 및 불필요한 column drop
df_all.reset_index(inplace=True)
df_all.drop(['index', 'Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)

In [11]:
df_all.isnull().sum()

Published         0
HDThumbnailUrl    0
ViewCount         0
VideoId           0
Title             0
dtype: int64

In [12]:
# @seah가 만든 labeling함수.
# 앞뒤 비교할 n개의 평균과 조회수를 비교하여 더 높으면 1, 낮으면 0을 칼럼에 저장한다.

def getLabels(df, n) :

    # input : dataset to get labels, number of neighbors to get mean of views
    # output : dataset with labels and mean of neighbors

    # create columns 'NeighborsMean' & 'label'
    df['NeighborsMean'] = 0
    df['label'] = 0
    df = df.astype({'NeighborsMean':'int'})

    for i in range(len(df)):
        mean = 0

        # [ : NUM_NEIGHBORS - 1 ]
        if i in range(0, n):
            mean = df.iloc[:i+n+1]['ViewCount'].mean()
        
        # [ NUM_NEIGHBORS : length - NUM_NEIGHBORS - 1 ]
        elif i in range(n, len(df)-n):
            mean = df.iloc[i-n:i+n+1]['ViewCount'].mean()
        
        # [ length - NUM_NEIGHBORS : ]
        else:
            mean = df.iloc[i-n:]['ViewCount'].mean()
        
        df['NeighborsMean'][i] = round(mean)
        if df['ViewCount'][i] >= mean:
            df['label'][i] = 1
    
    return df

In [13]:
df_all = getLabels(df_all, NUM_NEIGHBORS)
df_all.drop(['Published', 'ViewCount', 'NeighborsMean'], axis=1, inplace=True)
df_all

,HDThumbnailUrl,VideoId,Title,label
0,https://i.ytimg.com/vi/1-lSq0Y7Sf8/hqdefault.jpg,1-lSq0Y7Sf8,한국 패션이 어떠냐고요? 패션업계에 있는 유럽인들이 말하는 한국인의 패션!,0
1,https://i.ytimg.com/vi/z5-PlPKWMXk/hqdefault.jpg,z5-PlPKWMXk,프랑스 여자가 한국 공장에서 일하며 외노자가 된 이유는?,1
2,https://i.ytimg.com/vi/NFd-5sNibZE/hqdefault.jpg,NFd-5sNibZE,한국 고시원에 살아본 에스토니아 사람 #shorts,0
3,https://i.ytimg.com/vi/VUnf9lCdg_w/hqdefault.jpg,VUnf9lCdg_w,뉴욕에서 한국어를 배운 특이한 이유 ㅋㅋ #shorts,0
4,https://i.ytimg.com/vi/wtFDFJuvXhA/hqdefault.jpg,wtFDFJuvXhA,등교길에 지하철 분식을 지나칠 수 없는 미국여자! 김떡순 오뎅까지 혼밥 먹방,0
...,...,...,...,...
25392,https://i.ytimg.com/vi/GVEdfxvmdzk/hqdefault.jpg,GVEdfxvmdzk,충충이와 김포 구제창고에서 쇼핑한 하루,0
25393,https://i.ytimg.com/vi/mTCWMkTE6EQ/hqdefault.jpg,mTCWMkTE6EQ,"혜진쓰 Vlog | 퇴근 후 장보기, 접어먹는 김밥, 충충이랑 결혼고사 보는 하루",1
25394,https://i.ytimg.com/vi/2KnXlDCZXKE/hqdefault.jpg,2KnXlDCZXKE,혜진쓰 Vlog | 카메라📷 개봉하고 충충이랑 감성 가구🛋 구경하기,0
25395,https://i.ytimg.com/vi/SKwNdLWygIo/hqdefault.jpg,SKwNdLWygIo,혜진쓰 Vlog | 🎁지니핑크 신상 컨셉촬영📷하고 PT받고 밥까지 해먹는 바쁜하루:),0


In [14]:
df_all['label'].value_counts()

0    16294
1     9103
Name: label, dtype: int64

In [15]:
df_all.to_csv("AllLabeledData.csv")